In [2]:
import pandas as pd
import numpy as np
from amplpy import AMPL, DataFrame

# Load in relevant data

Note: CSP = Constraint Satisfaction Problem

In [3]:
responses = pd.read_csv("autoscheduler_Export.csv")
shifts = pd.read_csv("shifts.csv")
recovery_shifts = pd.read_csv('recoveryshifts.csv')
hours = responses[['Full Name', 'Volunteer Weekly Commitment']].set_index('Full Name')

In [4]:
print('responses:')
display(responses.head(2))
print('shifts')
display(shifts.head(2))
print('recovery_shifts')
display(recovery_shifts.head(2))
print('hours')
display(hours.head(2))

responses:


,Full Name,Volunteer Weekly Commitment,All Pantry Shift Availabilities,Recovery Shift Availabilities
0,Gillian Edgelow,2,"FRI 10-12AM,FRI 11-1PM,FRI 12-2PM,SAT 10-12AM,...",NaN
1,Giulia de Azeredo Valdejao,1,"THU 1-3PM,THU 2-4PM,MON 10-12PM",NaN


shifts


,Volunteer Shift,All Available Pantry Volunteers
0,MON 10-12PM,"Julian Kuzdovich,Danny Cao,Giulia Oltranti,Sal..."
1,TUE 9-11AM,"Andrea Ponce Mata,Serene Chang,Ed Hoffmann,Tas..."


recovery_shifts


,Volunteer Shift,Food Recovery Volunteers
0,MON 10-12PM,"Genna Fudin,Pascale Montgomery"
1,TUE 10-12PM,Isaiah Gallegos


hours


,Volunteer Weekly Commitment
Full Name,
Gillian Edgelow,2
Giulia de Azeredo Valdejao,1


## Transform data from volunteer form so it's usable by the CSP solver.

In [5]:
def Transform_Responses(response_df):
    all_shifts = list(shifts['Volunteer Shift'])
    columns = np.append(['Full Name'], all_shifts)
    availability = pd.DataFrame(columns=columns)
    for i in np.arange(response_df.shape[0]):
        mapping = []
        for j in all_shifts:
            if j in response_df.iloc[i,2]:
                mapping.append(1)
            else:
                mapping.append(0)
            row_data = np.append(response_df.iloc[i,0], mapping)
        availability.loc[len(availability)] = row_data
    return availability

In [6]:
availability = Transform_Responses(responses).set_index('Full Name')

# Run Constraint Satisfaction Algorithm

Variables: user-day-shift
Constraints:
+ 1 <= sum(shift) <= 5
+ 1 <= sum(volunteer) <= volunteer weekly commitment
+ ex. sum(user-mon-7-8) <= 5
+ 2 <= sum(last shift for day) <= 5 (should add ?)

In [7]:
import constraint as csp
#problem = Problem()
#problem.addVariables(["a", "b"], [1, 2, 3])
#problem.addConstraint(lambda a, b: b == a+1, ["a", "b"])
#solutions = problem.getSolutions()

## Create a couple helper functions for the CSP solver based on above constraints

In [8]:
def eqOne(*arg):
    return sum(arg) == 1

def greqN(x):
    def greq_helper(*arg):
        return sum(arg) >= x
    return greq_helper

def leeqN(x):
    def leeq_helper(*arg):
        return sum(arg) <= x
    return leeq_helper

## Define all decision variabls in form 'volunteer;shift'

In [11]:
all_shifts = list(shifts['Volunteer Shift'])
all_volunteers = list(responses['Full Name'])
variables = []
variable_domain = [0, 1]
for s in all_shifts:
    for v in all_volunteers:
        if int(availability.loc[v,s]) == 1:
            variables.append(v + ';' + s)

In [12]:
shift_Sum = []
for i in all_shifts:
    one_shift = []
    for j in variables:
        if j.split(';')[1] in i:
            one_shift.append(j)
    shift_Sum.append(one_shift)

In [13]:
vol_Sum = []
for i in all_volunteers:
    one_user = []
    for j in variables:
        if j.split(';')[0] in i:
            one_user.append(j)
    vol_Sum.append(one_user)

#### Main algorithm: way too slow (DON'T RUN)

In [53]:
problem = csp.Problem()
problem.addVariables(variables, variable_domain)
#constraints based on volunteer hours (volunteer shifts)
#lower bound of 1 on each volunteer (to ensure every volunteer has one shift):
for vs in vol_Sum:
    problem.addConstraint(greqN(0), vs)
#upper bound constraint for each user (volunteer weekly commitment)
for vs in vol_Sum:
    vol = vs[0].split(';')[0]
    k = hours.loc[vol][0]
    problem.addConstraint(leeqN(k), vs)

print('volunteer constraints created!')

#shift based constraints (filling each shift but not overfilling)
#lower bound (of 1) on shifts
for shift in shift_Sum:
    problem.addConstraint(greqN(0), shift)
    problem.addConstraint(leeqN(5), shift)
    
print('shift constraints created!')
    
#Get solution
solutions = problem.getSolution()

AttributeError: 'IntVar' object has no attribute 'split'

#### Simpler algorithm and returns one solution

## Google OR-Tools CSP solver

In [54]:
from ortools.sat.python import cp_model

## Model with maximize #assignments

In [77]:
model = cp_model.CpModel()

gvars = list()
for var in variables:
    var = model.NewIntVar(0, 1, var)
    gvars.append(var)

vol_Sum = []
for i in all_volunteers:
    one_user = []
    for j in gvars:
        if j.Name().split(';')[0] in i:
            one_user.append(j)
    vol_Sum.append(one_user)
    
shift_Sum = []
for i in all_shifts:
    one_shift = []
    for j in gvars:
        if j.Name().split(';')[1] in i:
            one_shift.append(j)
    shift_Sum.append(one_shift)
    
#constraints based on volunteer hours (volunteer shifts)
#lower bound of 1 on each volunteer (to ensure every volunteer has one shift):
for vs in vol_Sum:
    model.Add(sum(vs) >= 1)
    

#upper bound constraint for each user (volunteer weekly commitment)
for vs in vol_Sum:
    vol = vs[0].Name().split(';')[0]
    k = hours.loc[vol][0]
    model.Add(sum(vs) <= k)

print('volunteer constraints created!')

#shift based constraints (filling each shift but not overfilling)
#lower bound (of 1) on shifts
for shift in shift_Sum:
    model.Add(sum(shift) >= 1)
    model.Add(sum(shift) <= 10)
    
    
print('shift constraints created!')

#Maximize number of volunteers working
model.Maximize(sum(gvars))
    
#Get solution
#solutions = problem.getSolution()
solver = cp_model.CpSolver()
status = solver.Solve(model)

volunteer constraints created!
shift constraints created!


In [78]:
count = 0
for var in gvars:
    count += solver.Value(var)

In [79]:
count

163

## Finding Multiple Solutions - no limits

In [69]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            2 + 2
            #print(1)
            #print('%s=%i' % (v, self.Value(v)), end=' ')
        return()

    def solution_count(self):
        return self.__solution_count

In [70]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(gvars)
status = solver.SearchForAllSolutions(model, solution_printer)

In [43]:
solver.StatusName()

'FEASIBLE'